# Interact with ERPNEXT database

In [1]:
site_dir = "/home/cole/madrid/sites"
db_name = "cole"
db = "test"

In [2]:
import os

current_dir = os.getcwd()
os.chdir(site_dir)

import frappe

frappe.connect(db, db_name=db_name)

In [3]:
import sys

sys.path.append("/home/cole/madrid/apps/inventory_tools/inventory_tools")
from inventory_tools.inventory_tools.doctype.warehouse_plan import warehouse_plan

# Explore

In [1]:
pl = frappe.get_doc("Pick List", "STO-PICK-2025-00001").as_dict()
pl

NameError: name 'frappe' is not defined

In [ ]:
# get items in pick list doc
[item["item_code"] for item in pl["locations"]]

['Flour', 'Butter', 'Sugar', 'Bayberry']

In [ ]:
frappe.get_doc("Item", "Boysenberry").as_dict()

{'name': 'Boysenberry',
 'owner': 'Administrator',
 'creation': datetime.datetime(2025, 3, 11, 8, 59, 27, 874138),
 'modified': datetime.datetime(2025, 3, 11, 8, 59, 27, 925458),
 'modified_by': 'Administrator',
 'docstatus': 0,
 'idx': 0,
 'naming_series': 'STO-ITEM-.YYYY.-',
 'item_code': 'Boysenberry',
 'item_name': 'Boysenberry',
 'item_group': 'Ingredients',
 'stock_uom': 'Pound',
 'disabled': 0,
 'allow_alternative_item': 0,
 'is_stock_item': 1,
 'has_variants': 0,
 'opening_stock': 0.0,
 'valuation_rate': 0.0,
 'standard_rate': 0.0,
 'is_fixed_asset': 0,
 'auto_create_assets': 0,
 'is_grouped_asset': 0,
 'asset_category': None,
 'asset_naming_series': None,
 'over_delivery_receipt_allowance': 0.0,
 'over_billing_allowance': 0.0,
 'image': None,
 'description': 'A juicy, dark purple berry with a balance of tartness and sweetness.',
 'brand': 'Ambrosia Pie Co',
 'shelf_life_in_days': 7,
 'end_of_life': datetime.date(2099, 12, 31),
 'default_material_request_type': 'Purchase',
 'va

In [ ]:
item_warehouses = frappe.get_all(
    "Bin", fields=["warehouse"], filters={"item_code": "Boysenberry"}
)
for wh in item_warehouses:
    x = frappe.get_doc("Warehouse", wh["warehouse"]).as_dict()["parent_warehouse"]
frappe.get_doc("Warehouse", "All Warehouses - APC").as_dict()

# 'parent_warehouse' == ''

{'name': 'All Warehouses - APC',
 'owner': 'Administrator',
 'creation': datetime.datetime(2025, 3, 11, 8, 58, 51, 919967),
 'modified': datetime.datetime(2025, 3, 11, 8, 58, 51, 919967),
 'modified_by': 'Administrator',
 'docstatus': 0,
 'idx': 0,
 'disabled': 0,
 'warehouse_path': None,
 'warehouse_name': 'All Warehouses',
 'is_group': 1,
 'parent_warehouse': '',
 'is_rejected_warehouse': 0,
 'account': None,
 'company': 'Ambrosia Pie Company',
 'email_id': None,
 'phone_no': None,
 'mobile_no': None,
 'address_line_1': None,
 'address_line_2': None,
 'city': None,
 'state': None,
 'pin': None,
 'warehouse_type': None,
 'default_in_transit_warehouse': None,
 'lft': 1,
 'rgt': 20,
 'old_parent': '',
 'warehouse_plan': None,
 'warehouse_plan_coordinates': None,
 'rotation': 0,
 'accessible_path_x': 0,
 'accessible_path_y': 0,
 'doctype': 'Warehouse'}

In [ ]:
[wh["name"] for wh in frappe.get_all("Warehouse Plan")]

['All Warehouses - CFC']

In [ ]:
# def get_root_warehouse(warehouse):
#     # Finds closest parent parent warehouse with a walkable floor plan, otherwise returns None
#     wh_plans = [wh["name"] for wh in frappe.get_all("Warehouse Plan")]
#     if warehouse in wh_plans:
#         if frappe.get_doc("Warehouse Plan", warehouse).as_dict()["matrix"] is not None:
#             return warehouse
#     parent_warehouse = frappe.get_doc("Warehouse", warehouse).as_dict()["parent_warehouse"]
#     if parent_warehouse == "":
#         return None
#     return get_root_warehouse(parent_warehouse)

# # Example usage:
# item_warehouses = frappe.get_all("Bin", fields=["warehouse"], filters={"item_code": "Boysenberry"})
# for wh in item_warehouses:
#     root_warehouse = get_root_warehouse(wh["warehouse"])
#     print(f"Root warehouse for {wh['warehouse']}: {root_warehouse}")

In [ ]:
# def get_root_warehouse(warehouse):
#     warehouse_doc = frappe.get_doc("Warehouse", warehouse).as_dict()
#     if warehouse_doc["parent_warehouse"] == "":
#         return warehouse_doc["name"]
#     return get_root_warehouse(warehouse_doc["parent_warehouse"])

## Pick List Functions

In [ ]:
def get_root_warehouse(warehouse):
    # Finds closest parent warehouse with a walkable floor plan; otherwise returns None
    wh_plans = [wh["name"] for wh in frappe.get_all("Warehouse Plan")]
    if warehouse in wh_plans:
        wp_doc = frappe.get_doc("Warehouse Plan", warehouse)
        if wp_doc.as_dict()["matrix"] is not None:
            return warehouse
    parent_warehouse = frappe.get_doc("Warehouse", warehouse).as_dict()[
        "parent_warehouse"
    ]
    if parent_warehouse == "":
        return None
    return get_root_warehouse(parent_warehouse)


def get_all_warehouses(items):
    result = []
    for item_code in items:
        bins = frappe.get_all(
            "Bin", fields=["warehouse"], filters={"item_code": item_code}
        )
        for b in bins:
            result.append(
                {
                    "item_code": item_code,
                    "item_warehouses": b["warehouse"],
                    "root_warehouse": get_root_warehouse(b["warehouse"]),
                }
            )
    return result


def get_item_qty(item_code, warehouse):
    qty = frappe.db.get_value(
        "Bin", {"item_code": item_code, "warehouse": warehouse}, "actual_qty"
    )
    return qty if qty else 0


def get_node(doc, method):
    # Get root warehouse for all items in doc
    items = [item["item_code"] for item in doc["locations"]]
    root_warehouses = [get_root_warehouse(w["warehouse"]) for w in doc["locations"]]
    all_same = all(wh == root_warehouses[0] for wh in root_warehouses)
    if all_same is True:
        root_warehouse = root_warehouses[0]

    # Get all item locations for items in doctype
    # Filter by root warehouse
    item_wh_list = list(
        filter(
            lambda d: d["root_warehouse"] == root_warehouse, get_all_warehouses(items)
        )
    )
    # Item, Warehouse (in root), Qty, When Created (individial or most recent modify?)
    item_wh_qty_list = list(
        map(
            lambda d: {
                "item": d["item"],
                "warehouse": d["item_warehouses"],
                "qty": get_item_qty(d["item"], d["item_warehouses"]),
            },
            item_wh_list,
        )
    )

    if method == "FIFO":
        pass
    elif method == "LIFO":
        pass
    elif method == "Deplete maximum number of Bins":
        pass
    elif method == "Deplete minimum number of Bins":
        pass
    elif method == "Shortest Path":
        pass
    return 1

In [ ]:
from erpnext.stock.report.stock_ageing.stock_ageing import FIFOSlots
from frappe.utils.data import nowdate


def get_root_warehouse(warehouse):
    # Finds closest parent warehouse with a walkable floor plan; otherwise returns None
    wh_plans = [wh["name"] for wh in frappe.get_all("Warehouse Plan")]
    if warehouse in wh_plans:
        wp_doc = frappe.get_doc("Warehouse Plan", warehouse)
        if wp_doc.as_dict()["matrix"] is not None:
            return warehouse
    parent_warehouse = frappe.get_doc("Warehouse", warehouse).as_dict()[
        "parent_warehouse"
    ]
    if parent_warehouse == "":
        return None
    return get_root_warehouse(parent_warehouse)


def get_all_warehouses(item_code):
    """Return all warehouses for a single item, including its root warehouse."""
    bins = frappe.get_all("Bin", fields=["warehouse"], filters={"item_code": item_code})
    return [
        {
            "item_code": item_code,
            "warehouse": b["warehouse"],
            "root_warehouse": get_root_warehouse(b["warehouse"]),
        }
        for b in bins
    ]


def get_item_qty(item_code, warehouse):
    """Return the actual quantity for the given item and warehouse."""
    qty = frappe.db.get_value(
        "Bin", {"item_code": item_code, "warehouse": warehouse}, "actual_qty"
    )
    return qty or 0


def get_bin_modified(item_code, warehouse):
    return frappe.db.get_value(
        "Bin",
        {"item_code": "Boysenberry", "warehouse": "Refrigerator - APC"},
        "modified",
    )


def get_node(doc, method):
    company = doc["company"]
    # Extract item codes and root warehouses from document locations
    itemdict = {}
    for loc in doc["locations"]:
        if itemdict.get(loc["item_code"]):
            itemdict[loc["item_code"]]["qty"] += loc["qty"]
        else:
            itemdict[loc["item_code"]] = {"qty": loc["qty"]}

    # Ensure all locations share the same root warehouse
    root_warehouses = [get_root_warehouse(loc["warehouse"]) for loc in doc["locations"]]
    if not all(wh == root_warehouses[0] for wh in root_warehouses):
        # Handle the case where the root warehouses differ as needed.
        return
    root_warehouse = root_warehouses[0]

    itemdict = {"Water": {"qty": 1000}}

    if method == "FIFO":

        nodes = []
        for item in itemdict:
            qty_req = itemdict[item]["qty"]
            filters = {
                "item_code": item,
                "company": company,
                "to_date": nowdate(),
            }
            fifoque = FIFOSlots(filters=filters).generate()
            qty_obt = 0
            for wh_entry_qty in fifoque[item]["fifo_queue"]:
                if (
                    get_root_warehouse(fifoque[item]["details"]["warehouse"])
                    != root_warehouse
                ):
                    continue

                if wh_entry_qty[0] > (qty_req - qty_obt):
                    nodes.append(
                        {
                            "item_code": item,
                            "warehouse": fifoque[item]["details"]["warehouse"],
                            "qty": qty_req - qty_obt,
                        }
                    )
                    break
                else:
                    nodes.append(
                        {
                            "item_code": item,
                            "warehouse": fifoque[item]["details"]["warehouse"],
                            "qty": wh_entry_qty[0],
                        }
                    )
    return nodes


get_node(pl, "FIFO")

[{'item_code': 'Water',
  'warehouse': 'Refrigerated Display - APC',
  'qty': 50.0},
 {'item_code': 'Water',
  'warehouse': 'Refrigerated Display - APC',
  'qty': 15.0},
 {'item_code': 'Water',
  'warehouse': 'Refrigerated Display - APC',
  'qty': 100.0}]

In [ ]:
pl

{'name': 'STO-PICK-2025-00001',
 'owner': 'Administrator',
 'creation': datetime.datetime(2025, 2, 27, 4, 5, 23, 785966),
 'modified': datetime.datetime(2025, 2, 27, 4, 5, 23, 984758),
 'modified_by': 'Administrator',
 'docstatus': 0,
 'idx': 0,
 'naming_series': 'STO-PICK-.YYYY.-',
 'company': 'Ambrosia Pie Company',
 'purpose': 'Delivery',
 'customer': 'Sand Street Deli',
 'customer_name': 'Sand Street Deli',
 'work_order': None,
 'material_request': None,
 'for_qty': 0.0,
 'parent_warehouse': None,
 'consider_rejected_warehouses': 0,
 'pick_manually': 0,
 'ignore_pricing_rule': 0,
 'scan_barcode': None,
 'scan_mode': 0,
 'prompt_qty': 0,
 'amended_from': None,
 'group_same_items': 0,
 'status': 'Draft',
 'doctype': 'Pick List',
 'locations': [{'name': 'vt8i09idg5',
   'owner': 'Administrator',
   'creation': datetime.datetime(2025, 2, 27, 4, 5, 23, 785966),
   'modified': datetime.datetime(2025, 2, 27, 4, 5, 23, 785966),
   'modified_by': 'Administrator',
   'docstatus': 0,
   'idx'

In [ ]:
# items = [item["item_code"] for item in pl["locations"]]

itemdict = {}
for loc in pl["locations"]:
    if itemdict.get(loc["item_code"]):
        itemdict[loc["item_code"]]["qty"] += loc["qty"]
    else:
        itemdict[loc["item_code"]] = {"qty": loc["qty"]}
# list(itemdict.keys())

# Filter List

In [45]:
root_warehouses = [get_root_warehouse(loc["warehouse"]) for loc in pl["locations"]]
all_same = all(wh == root_warehouses[0] for wh in root_warehouses)
root_warehouse = root_warehouses[0]
all_same

True

In [46]:
frappe.db.get_value(
    "Bin", {"item_code": "Boysenberry", "warehouse": "Refrigerator - APC"}, "actual_qty"
)

94.0

In [47]:
frappe.get_doc("Bin", "k1p03s9jno").as_dict()

{'name': 'k1p03s9jno',
 'owner': 'Administrator',
 'creation': datetime.datetime(2025, 2, 26, 5, 56, 11, 395279),
 'modified': datetime.datetime(2025, 2, 26, 5, 56, 11, 402750),
 'modified_by': 'Administrator',
 'docstatus': 0,
 'idx': 0,
 'item_code': 'Double Plum Pie Filling',
 'warehouse': 'Refrigerator - APC',
 'actual_qty': 0.0,
 'planned_qty': 0.0,
 'indented_qty': 0.0,
 'ordered_qty': 0.0,
 'projected_qty': -120.0,
 'reserved_qty': 0.0,
 'reserved_qty_for_production': 120.0,
 'reserved_qty_for_sub_contract': 0.0,
 'reserved_qty_for_production_plan': 0.0,
 'reserved_stock': 0.0,
 'stock_uom': 'Cup',
 'valuation_rate': 0.0,
 'stock_value': 0.0,
 'doctype': 'Bin'}

In [48]:
# Add modified
def get_warehouse_modified(item_code, warehouse):
    return frappe.db.get_value(
        "Bin",
        {"item_code": "Boysenberry", "warehouse": "Refrigerator - APC"},
        "modified",
    )

# Methods

In [1]:
pl

NameError: name 'pl' is not defined

## Max Bins

In [ ]:
sle = frappe.get_all(
    "Stock Ledger Entry",
    fields=["actual_qty", "posting_date", "creation", "warehouse"],
    filters={
        "item_code": "Water",
        "company": "Ambrosia Pie Company",
        "posting_date": ["<=", nowdate()],
        "is_cancelled": 0,
        "actual_qty": [">", 0],
    },
    order_by="actual_qty, posting_date, creation",
)
sle

[{'actual_qty': 15.0,
  'posting_date': datetime.date(2025, 1, 1),
  'creation': datetime.datetime(2025, 2, 26, 5, 56, 20, 717796),
  'warehouse': 'Kitchen - APC'},
 {'actual_qty': 50.0,
  'posting_date': datetime.date(2025, 1, 1),
  'creation': datetime.datetime(2025, 2, 26, 5, 55, 55, 475215),
  'warehouse': 'Kitchen - APC'},
 {'actual_qty': 100.0,
  'posting_date': datetime.date(2025, 3, 6),
  'creation': datetime.datetime(2025, 3, 6, 7, 8, 13, 312590),
  'warehouse': 'Refrigerated Display - APC'}]

## FIFO Scratch

In [ ]:
from erpnext.stock.report.stock_ageing.stock_ageing import FIFOSlots
from frappe.utils.data import nowdate

filters = {
    "item_code": "Water",
    "company": "Ambrosia Pie Company",
    "to_date": nowdate(),
}


fifoque = FIFOSlots(filters=filters).generate()
fifoque
# print(fifoque)
# fifoque["Kakadu Lime"]["fifo_queue"][0][0]
# fifoque["Kakadu Lime"]["details"]["warehouse"]

{'Water': {'details': {'name': 'Water',
   'item_name': 'Water',
   'item_group': 'Ingredients',
   'brand': 'Ambrosia Pie Co',
   'description': 'Water',
   'stock_uom': 'Cup',
   'has_serial_no': 0,
   'actual_qty': 100.0,
   'posting_date': datetime.date(2025, 3, 6),
   'voucher_type': 'Stock Entry',
   'voucher_no': 'MAT-STE-2025-00005',
   'serial_no': None,
   'batch_no': None,
   'qty_after_transaction': 100.0,
   'serial_and_batch_bundle': None,
   'warehouse': 'Refrigerated Display - APC'},
  'fifo_queue': [[50.0, datetime.date(2025, 1, 1)],
   [15.0, datetime.date(2025, 1, 1)],
   [100.0, datetime.date(2025, 3, 6)]],
  'qty_after_transaction': 165.0,
  'total_qty': 165.0,
  'has_serial_no': 0}}

In [ ]:
from frappe.utils import nowdate, get_datetime
import frappe
from erpnext.stock.report.stock_ageing.stock_ageing import FIFOSlots

# Define your filters
filters = {
    "item_code": "Water",
    "company": "Ambrosia Pie Company",
    "to_date": nowdate(),  # Today's date, for example
}

# Build a query for Stock Ledger Entries using frappe.qb
sle = frappe.qb.DocType("Stock Ledger Entry")
item = frappe.qb.DocType("Item")
to_date = get_datetime(filters.get("to_date") + " 23:59:59")

query = (
    frappe.qb.from_(sle)
    .from_(item)
    .select(
        item.name,
        item.item_name,
        item.item_group,
        item.brand,
        item.description,
        item.stock_uom,
        item.has_serial_no,
        item.valuation_method,
        sle.actual_qty,
        sle.stock_value_difference,
        sle.valuation_rate,
        sle.posting_date,
        sle.voucher_type,
        sle.voucher_no,
        sle.serial_no,
        sle.batch_no,
        sle.qty_after_transaction,
        sle.serial_and_batch_bundle,
        sle.warehouse,
    )
    .where(
        (sle.item_code == item.name)
        & (sle.company == filters.get("company"))
        & (sle.posting_datetime <= to_date)
        & (sle.is_cancelled != 1)
    )
)

# Execute the query and get SLEs as a list of dictionaries
results = query.run(as_dict=True)

# Pass the SLE results to FIFOSlots
fifoque = FIFOSlots(filters=filters, sle=results).generate()
fifoque["Water"]

{'details': {'name': 'Water',
  'item_name': 'Water',
  'item_group': 'Ingredients',
  'brand': 'Ambrosia Pie Co',
  'description': 'Water',
  'stock_uom': 'Cup',
  'has_serial_no': 0,
  'valuation_method': 'Moving Average',
  'actual_qty': 100.0,
  'stock_value_difference': 100.0,
  'valuation_rate': 1.0,
  'posting_date': datetime.date(2025, 3, 6),
  'voucher_type': 'Stock Entry',
  'voucher_no': 'MAT-STE-2025-00005',
  'serial_no': None,
  'batch_no': None,
  'qty_after_transaction': 100.0,
  'serial_and_batch_bundle': None,
  'warehouse': 'Refrigerated Display - APC'},
 'fifo_queue': [[50.0, datetime.date(2025, 1, 1)],
  [15.0, datetime.date(2025, 1, 1)],
  [100.0, datetime.date(2025, 3, 6)]],
 'qty_after_transaction': 165.0,
 'total_qty': 165.0,
 'has_serial_no': 0}

In [ ]:
from erpnext.stock.utils import get_latest_stock_qty, get_latest_stock_balance

get_latest_stock_balance()

{'Refrigerator - APC': {'Kandis': 0.0,
  'Batuan': 0.0,
  'Eugenia Uniflora': 0.0,
  'Bacuri': 0.0,
  'White Currant': 0.0,
  'Sultana': 0.0,
  'Durian': 0.0,
  'Banana': 0.0,
  'Bayberry Pie Filling': 0.0,
  'Hackberry': 0.0,
  'Date': 0.0,
  'European Blueberry': 0.0,
  'Bilimbi': 0.0,
  'Cryptocarya Alba': 0.0,
  'Bearberry': 0.0,
  'Kahikatea': 0.0,
  'Redcurrant': 0.0,
  'Lime': 0.0,
  'Photinia': 0.0,
  'Triphasia Brassii': 0.0,
  'Raisin': 0.0,
  'Olallieberry': 0.0,
  'Candlenut': 0.0,
  'Cowberry': 0.0,
  'Fragaria': 0.0,
  'Mini Watermelon': 0.0,
  'Black Cherry': 0.0,
  'Raspberry': 0.0,
  'Empetrum': 0.0,
  'Louvi': 0.0,
  'Gooseberry Pie Filling': 0.0,
  'Barbados Cherry': 0.0,
  'Highbush Cranberry': 0.0,
  'Cloudberry': 0.0,
  'Kaduka Lime': 0.0,
  'Guava': 0.0,
  'Jostaberry': 0.0,
  'Dabai': 0.0,
  'Black Apple': 0.0,
  'Huckleberry': 0.0,
  'Bacupari': 0.0,
  'Midyim': 0.0,
  'Boquila': 0.0,
  'Goji': 0.0,
  'Kapok': 0.0,
  'Bitter Melon': 0.0,
  'Desert Banana': 0.0,

In [ ]:
import frappe
from frappe.utils import nowdate


def get_fifo_allocation(
    item_code: str, company: str, to_date: str, required_qty: float
):
    """
    Returns FIFO allocation for a given item as a list of dicts:
    Each dict contains:
        - 'item_code'
        - 'qty' : quantity taken from that FIFO bin
        - 'warehouse' : warehouse from which stock was allocated

    Uses frappe.get_all to fetch SLE records in FIFO order.
    """
    # Query positive (incoming) SLE records for the given item using the ORM
    bins = frappe.get_all(
        "Stock Ledger Entry",
        fields=[
            "actual_qty",
            "valuation_rate",
            "posting_date",
            "creation",
            "warehouse",
        ],
        filters={
            "item_code": item_code,
            "company": company,
            "posting_date": ["<=", to_date],
            "is_cancelled": 0,
            "actual_qty": [">", 0],
        },
        order_by="posting_date, creation",
    )

    allocation = []
    remaining = required_qty

    # Process each incoming stock bin in FIFO order
    for bin in bins:
        if remaining <= 0:
            break

        available = bin.get("actual_qty", 0)
        allocated_qty = min(available, remaining)
        allocation.append(
            {
                "item_code": item_code,
                "qty": allocated_qty,
                "warehouse": bin.get("warehouse"),
            }
        )
        remaining -= allocated_qty

    if remaining > 0:
        frappe.msgprint(f"Warning: Not enough stock. Short by {remaining}")

    return allocation


# --- Example Usage ---
item_code = "Water"
company = "Ambrosia Pie Company"
to_date = nowdate()  # or specify a date string (e.g., "2025-03-06")
required_qty = 150  # specify the consumption quantity

fifo_allocation = get_fifo_allocation(item_code, company, to_date, required_qty)
print("FIFO Allocation:", fifo_allocation)

FIFO Allocation: [{'item_code': 'Water', 'qty': 50.0, 'warehouse': 'Kitchen - APC'}, {'item_code': 'Water', 'qty': 15.0, 'warehouse': 'Kitchen - APC'}, {'item_code': 'Water', 'qty': 85.0, 'warehouse': 'Refrigerated Display - APC'}]


In [ ]:
# v2 final final final

sle = frappe.get_all(
    "Stock Ledger Entry",
    fields=["actual_qty", "posting_date", "creation", "warehouse"],
    filters={
        "item_code": "Water",
        "company": "Ambrosia Pie Company",
        "posting_date": ["<=", nowdate()],
        "is_cancelled": 0,
        "actual_qty": [">", 0],
    },
    order_by="posting_date, creation",
)
sle

[{'actual_qty': 50.0,
  'posting_date': datetime.date(2025, 1, 1),
  'creation': datetime.datetime(2025, 2, 26, 5, 55, 55, 475215),
  'warehouse': 'Kitchen - APC'},
 {'actual_qty': 15.0,
  'posting_date': datetime.date(2025, 1, 1),
  'creation': datetime.datetime(2025, 2, 26, 5, 56, 20, 717796),
  'warehouse': 'Kitchen - APC'},
 {'actual_qty': 100.0,
  'posting_date': datetime.date(2025, 3, 6),
  'creation': datetime.datetime(2025, 3, 6, 7, 8, 13, 312590),
  'warehouse': 'Refrigerated Display - APC'}]

## FIFO final final v2

In [ ]:
class Rules:
    @staticmethod
    def _process_entries(item_code, qty, company, order_by, root_warehouse, to_date):
        # Retrieve stock ledger entries with the provided filters and ordering.
        sle = frappe.get_all(
            "Stock Ledger Entry",
            fields=["actual_qty", "posting_date", "creation", "warehouse"],
            filters={
                "item_code": item_code,
                "company": company,
                "posting_date": ["<=", to_date],
                "is_cancelled": 0,
                "actual_qty": [">", 0],
            },
            order_by=order_by,
        )

        newsle = []
        qty_obtained = 0

        # Process each entry until the required quantity is fulfilled.
        for entry in sle:
            # If a root warehouse is specified, ensure the entry belongs to it.
            if (
                root_warehouse
                and get_root_warehouse(entry["warehouse"]) != root_warehouse
            ):
                continue

            remaining_qty = qty - qty_obtained

            if entry["actual_qty"] > remaining_qty:
                newsle.append(
                    {
                        "item_code": item_code,
                        "warehouse": entry["warehouse"],
                        "qty": remaining_qty,
                    }
                )
                qty_obtained += remaining_qty
                break
            else:
                newsle.append(
                    {
                        "item_code": item_code,
                        "warehouse": entry["warehouse"],
                        "qty": entry["actual_qty"],
                    }
                )
                qty_obtained += entry["actual_qty"]

        # If the accumulated quantity doesn't match the requested quantity, raise an error.
        if (qty - qty_obtained) != 0:
            raise frappe.ValidationError("Not enough items in root warehouse")
        return newsle

    @staticmethod
    def FIFO(item_code, qty, company, root_warehouse=None, to_date=nowdate()):
        # FIFO: Order by posting_date and creation in ascending order.
        return Rules._process_entries(
            item_code, qty, company, "posting_date, creation", root_warehouse, to_date
        )

    @staticmethod
    def LIFO(item_code, qty, company, root_warehouse=None, to_date=nowdate()):
        # LIFO: Order by posting_date and creation in descending order.
        return Rules._process_entries(
            item_code,
            qty,
            company,
            "posting_date desc, creation desc",
            root_warehouse,
            to_date,
        )

In [ ]:
Rules.FIFO("Water", 100, "Ambrosia Pie Company", "All Warehouses - APC")

[{'item_code': 'Water', 'warehouse': 'Kitchen - APC', 'qty': 50.0},
 {'item_code': 'Water', 'warehouse': 'Kitchen - APC', 'qty': 15.0},
 {'item_code': 'Water',
  'warehouse': 'Refrigerated Display - APC',
  'qty': 35.0}]

In [ ]:
pl

{'name': 'STO-PICK-2025-00001',
 'owner': 'Administrator',
 'creation': datetime.datetime(2025, 2, 27, 4, 5, 23, 785966),
 'modified': datetime.datetime(2025, 2, 27, 4, 5, 23, 984758),
 'modified_by': 'Administrator',
 'docstatus': 0,
 'idx': 0,
 'naming_series': 'STO-PICK-.YYYY.-',
 'company': 'Ambrosia Pie Company',
 'purpose': 'Delivery',
 'customer': 'Sand Street Deli',
 'customer_name': 'Sand Street Deli',
 'work_order': None,
 'material_request': None,
 'for_qty': 0.0,
 'parent_warehouse': None,
 'consider_rejected_warehouses': 0,
 'pick_manually': 0,
 'ignore_pricing_rule': 0,
 'scan_barcode': None,
 'scan_mode': 0,
 'prompt_qty': 0,
 'amended_from': None,
 'group_same_items': 0,
 'status': 'Draft',
 'doctype': 'Pick List',
 'locations': [{'name': 'vt8i09idg5',
   'owner': 'Administrator',
   'creation': datetime.datetime(2025, 2, 27, 4, 5, 23, 785966),
   'modified': datetime.datetime(2025, 2, 27, 4, 5, 23, 785966),
   'modified_by': 'Administrator',
   'docstatus': 0,
   'idx'

# Testing

In [4]:
from inventory_tools.inventory_tools.overrides.pick_list import (
    optimize_path,
    optimize_route_picklist,
)
from pprint import pprint

# Stock for the following items:
# {'Tower of Bay-bel': 'MFG-WO-2025-00005',
#  'Pocketful of Bay': 'MFG-WO-2025-00006',
#  'Bayberry Pie': 'MFG-WO-2025-00012',
#  'Bayberry Pocket': 'MFG-WO-2025-00014',
#  'Bayberry Popper': 'MFG-WO-2025-00015'}

# Bayberry popper
# Create work order: MFG-WO-2025-00017
# Create pick list: STO-PICK-2025-00002
pl = frappe.get_doc("Pick List", "STO-PICK-2025-00002").as_dict()
grid = [
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
]
save_grid = str(grid)
whplan = frappe.get_doc("Warehouse Plan", "All Warehouses - CFC")
whplan.update({"matrix": save_grid})
whplan.save()
whplan.submit()

<WarehousePlan: All Warehouses - CFC docstatus=1>

In [5]:
methods = [
    "FIFO",
    "LIFO",
    "Deplete maximum number of Bins",
    "Deplete minimum number of Bins",
]
for m in methods:
    print(f"Method: {m}")
    # out = optimize_path(pl, m)
    out = optimize_path("STO-PICK-2025-00003", m)
    pprint(out)

Method: FIFO
[{'item_code': 'Bayberry', 'qty': 6.0, 'warehouse': 'Fruit Basket 72 - CFC'},
 {'item_code': 'Bayberry', 'qty': 4.0, 'warehouse': 'Fruit Basket 80 - CFC'},
 {'item_code': 'Butter', 'qty': 1.0, 'warehouse': 'Fruit Basket 41 - CFC'},
 {'item_code': 'Butter', 'qty': 1.0, 'warehouse': 'Fruit Basket 44 - CFC'},
 {'item_code': 'Butter', 'qty': 0.5, 'warehouse': 'Fruit Basket 52 - CFC'},
 {'item_code': 'Sugar', 'qty': 0.5, 'warehouse': 'Fruit Basket 21 - CFC'},
 {'item_code': 'Flour', 'qty': 5.0, 'warehouse': 'Fruit Basket 32 - CFC'}]
Method: LIFO
[{'item_code': 'Sugar', 'qty': 0.5, 'warehouse': 'Fruit Basket 30 - CFC'},
 {'item_code': 'Bayberry', 'qty': 6.0, 'warehouse': 'Fruit Basket 69 - CFC'},
 {'item_code': 'Bayberry', 'qty': 3.0, 'warehouse': 'Fruit Basket 74 - CFC'},
 {'item_code': 'Bayberry', 'qty': 1.0, 'warehouse': 'Fruit Basket 62 - CFC'},
 {'item_code': 'Butter', 'qty': 1.5, 'warehouse': 'Fruit Basket 63 - CFC'},
 {'item_code': 'Butter', 'qty': 1.0, 'warehouse': 'Frui

In [6]:
out

[{'item_code': 'Flour', 'warehouse': 'Fruit Basket 32 - CFC', 'qty': 5.0},
 {'item_code': 'Bayberry', 'warehouse': 'Fruit Basket 71 - CFC', 'qty': 10.0},
 {'item_code': 'Butter', 'warehouse': 'Fruit Basket 63 - CFC', 'qty': 2.5},
 {'item_code': 'Sugar', 'warehouse': 'Fruit Basket 38 - CFC', 'qty': 0.5}]

In [7]:
optimize_route_picklist(out, "All Warehouses - CFC")

[{'item_code': 'Flour', 'warehouse': 'Fruit Basket 32 - CFC', 'qty': 5.0},
 {'item_code': 'Bayberry', 'warehouse': 'Fruit Basket 71 - CFC', 'qty': 10.0},
 {'item_code': 'Butter', 'warehouse': 'Fruit Basket 63 - CFC', 'qty': 2.5},
 {'item_code': 'Sugar', 'warehouse': 'Fruit Basket 38 - CFC', 'qty': 0.5}]

In [8]:
# Get Grid
import numpy as np
from frappe.utils import safe_json_loads

root_warehouse = "All Warehouses - CFC"
grid = np.array(
    safe_json_loads(
        frappe.get_doc("Warehouse Plan", root_warehouse).as_dict()["matrix"]
    )
)
grid

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1],
       [1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
        1, 1],
       [1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
        1, 1],
       [1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
        1, 1],
       [1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
        1, 1],
       [1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
        1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0],
       [0, 1, 1, 1, 1

In [9]:
# get scale
import numpy as np

imaginary_x = grid.shape[1]

real_x = frappe.get_doc("Warehouse Plan", root_warehouse).as_dict()["horizontal"]

scale = real_x / imaginary_x
scale

1.0

In [10]:
frappe.get_doc("Warehouse", "Fruit Basket 41 - CFC").as_dict()

{'name': 'Fruit Basket 41 - CFC',
 'owner': 'Administrator',
 'creation': datetime.datetime(2025, 3, 12, 7, 18, 10, 714287),
 'modified': datetime.datetime(2025, 3, 12, 7, 18, 10, 729402),
 'modified_by': 'Administrator',
 'docstatus': 1,
 'idx': 0,
 'disabled': 0,
 'warehouse_path': None,
 'warehouse_name': 'Fruit Basket 41',
 'is_group': 0,
 'parent_warehouse': 'All Warehouses - CFC',
 'is_rejected_warehouse': 0,
 'account': None,
 'company': 'Chelsea Fruit Co',
 'email_id': None,
 'phone_no': None,
 'mobile_no': None,
 'address_line_1': None,
 'address_line_2': None,
 'city': None,
 'state': None,
 'pin': None,
 'warehouse_type': None,
 'default_in_transit_warehouse': None,
 'lft': 112,
 'rgt': 113,
 'old_parent': 'All Warehouses - CFC',
 'warehouse_plan': None,
 'warehouse_plan_coordinates': None,
 'rotation': 0,
 'accessible_path_x': 10,
 'accessible_path_y': 11,
 'doctype': 'Warehouse'}

In [11]:
out

[{'item_code': 'Flour', 'warehouse': 'Fruit Basket 32 - CFC', 'qty': 5.0},
 {'item_code': 'Bayberry', 'warehouse': 'Fruit Basket 71 - CFC', 'qty': 10.0},
 {'item_code': 'Butter', 'warehouse': 'Fruit Basket 63 - CFC', 'qty': 2.5},
 {'item_code': 'Sugar', 'warehouse': 'Fruit Basket 38 - CFC', 'qty': 0.5}]

In [12]:
# get waypoints
item_whs = out
unique_whs = list({item_wh["warehouse"] for item_wh in item_whs})
waypoints = []
for wh in unique_whs:
    loc = frappe.get_doc("Warehouse", wh).as_dict()
    waypoints.append({wh: (loc["accessible_path_x"], loc["accessible_path_y"])})
waypoints

[{'Fruit Basket 63 - CFC': (15, 4)},
 {'Fruit Basket 38 - CFC': (9, 5)},
 {'Fruit Basket 32 - CFC': (7, 14)},
 {'Fruit Basket 71 - CFC': (18, 1)}]